## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-28-01-00-52 +0000,cbc,Over 200 candidates sign up for Poilievre's by...,https://www.cbc.ca/news/politics/over-200-cand...
1,2025-07-28-01-00-00 +0000,wsj,CEOs Trumpet Smaller Workforces as a Sign of C...,https://www.wsj.com/lifestyle/careers/layoff-b...
2,2025-07-28-01-00-00 +0000,wsj,A Tiny Company Is Vouching for Risky Insurers ...,https://www.wsj.com/finance/small-insurance-co...
3,2025-07-28-01-00-00 +0000,wsj,Individual home buyers are largely locked out ...,https://www.wsj.com/economy/housing/with-indiv...
4,2025-07-28-00-45-42 +0000,bbc,The rise of Japan's far right was supercharged...,https://www.bbc.com/news/articles/cx2k29233jeo


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2471/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
38,trump,28
178,gaza,14
62,new,10
244,aid,10
87,trade,8


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
163,2025-07-27-07-44-36 +0000,nypost,Thailand and Cambodia agree to cease-fire talk...,https://nypost.com/2025/07/27/world-news/thail...,72
100,2025-07-27-17-13-12 +0000,nypost,Trump says ‘Israel is gonna have to make a dec...,https://nypost.com/2025/07/27/us-news/trump-sa...,70
169,2025-07-27-04-07-59 +0000,nyt,Trump Says He Expects Quick Cease-Fire Between...,https://www.nytimes.com/2025/07/26/world/asia/...,64
174,2025-07-27-03-14-00 +0000,wsj,Thailand and Cambodia Consider Cease-Fire Afte...,https://www.wsj.com/world/asia/thailand-and-ca...,63
96,2025-07-27-17-37-25 +0000,nypost,Trump strikes ‘biggest deal ever made’ with EU...,https://nypost.com/2025/07/27/us-news/trump-eu...,62


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
163,72,2025-07-27-07-44-36 +0000,nypost,Thailand and Cambodia agree to cease-fire talk...,https://nypost.com/2025/07/27/world-news/thail...
100,42,2025-07-27-17-13-12 +0000,nypost,Trump says ‘Israel is gonna have to make a dec...,https://nypost.com/2025/07/27/us-news/trump-sa...
50,42,2025-07-27-21-25-01 +0000,startribune,US-EU deal sets a 15% tariff on most goods and...,https://www.startribune.com/trumps-meeting-wit...
7,35,2025-07-28-00-23-00 +0000,nypost,Thousands of New Yorkers — including ‘Orange i...,https://nypost.com/2025/07/27/us-news/thousand...
38,26,2025-07-27-22-21-33 +0000,nypost,Disturbing video shows woman knocked out cold ...,https://nypost.com/2025/07/27/us-news/disturbi...
16,25,2025-07-27-23-44-06 +0000,nypost,"California mom-of-four Whisper Owen, baby go m...",https://nypost.com/2025/07/27/us-news/californ...
51,24,2025-07-27-21-24-50 +0000,nyt,Suspect in Traverse City Walmart Stabbing Atta...,https://www.nytimes.com/2025/07/27/us/walmart-...
64,24,2025-07-27-20-30-26 +0000,nypost,At least 3 killed and others injured in train ...,https://nypost.com/2025/07/27/world-news/at-le...
36,23,2025-07-27-22-24-26 +0000,nypost,Hundreds of Gen Zers attempt world record at l...,https://nypost.com/2025/07/27/us-news/hundreds...
33,22,2025-07-27-22-28-40 +0000,nypost,AOC ripped as her NYC district is overrun by s...,https://nypost.com/2025/07/27/us-news/aoc-ripp...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
